In [1]:
import sqlite3
conn = sqlite3.connect("database.sqlite")
cur = conn.cursor()

In [2]:
import pandas as pd 
cur.execute('''SELECT * FROM matches''')
df = pd.DataFrame(cur.fetchall())
df.columns = (x[0] for x in cur.description)

In [3]:
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


# Season 2011

In [5]:
cur.execute("""Select *
               From Matches
               Where Season == '2011'
               and  Div != 'E0'
;""")
df1 = pd.DataFrame(cur.fetchall())
df1.columns = [x[0] for x in cur.description]
df1.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


# Total Home Goals

In [6]:
cur.execute("""Select Season , Date , HomeTeam  ,Div, FTR as Result, sum(FTHG) As Home_Goals
               From Matches
               Where Season == '2011'
               and  Div != 'E0'
               Group By HomeTeam
;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Season,Date,HomeTeam,Div,Result,Home_Goals
0,2011,2012-03-30,Aachen,D2,A,15
1,2011,2011-11-06,Augsburg,D1,A,20
2,2011,2011-11-19,Bayern Munich,D1,A,49
3,2011,2012-03-02,Bochum,D2,A,23
4,2011,2011-10-22,Braunschweig,D2,A,21


In [7]:
df.Result.unique()

array(['A', 'H', 'D'], dtype=object)

In [8]:
df['HomeWin'] = df['Result'].map(lambda x: 1 if x == 'H' else 0)

In [9]:
df.head()

,Season,Date,HomeTeam,Div,Result,Home_Goals,HomeWin
0,2011,2012-03-30,Aachen,D2,A,15,0
1,2011,2011-11-06,Augsburg,D1,A,20,0
2,2011,2011-11-19,Bayern Munich,D1,A,49,0
3,2011,2012-03-02,Bochum,D2,A,23,0
4,2011,2011-10-22,Braunschweig,D2,A,21,0


# Total Away Goals

In [10]:
cur.execute("""Select Season , Date , AwayTeam  ,Div, FTR as Result, sum(FTAG) As Away_Goals
               From Matches
               Where Season == '2011'
               and  Div != 'E0'
               Group By AwayTeam
;""")
df1 = pd.DataFrame(cur.fetchall())
df1.columns = [x[0] for x in cur.description]
df1.head()

,Season,Date,AwayTeam,Div,Result,Away_Goals
0,2011,2011-11-20,Aachen,D2,H,15
1,2011,2011-08-27,Augsburg,D1,H,16
2,2011,2012-03-31,Bayern Munich,D1,A,28
3,2011,2011-11-27,Bochum,D2,H,18
4,2011,2011-11-18,Braunschweig,D2,H,16


In [11]:
df1['AwayWin'] = df1['Result'].map(lambda x: 1 if x == 'A' else 0)
df1.head()

,Season,Date,AwayTeam,Div,Result,Away_Goals,AwayWin
0,2011,2011-11-20,Aachen,D2,H,15,0
1,2011,2011-08-27,Augsburg,D1,H,16,0
2,2011,2012-03-31,Bayern Munich,D1,A,28,1
3,2011,2011-11-27,Bochum,D2,H,18,0
4,2011,2011-11-18,Braunschweig,D2,H,16,0


# Joining Tables for Total Goals 

In [12]:
goals = pd.concat([df,df1], axis=1, join='inner')

In [13]:
goals.head()

,Season,Date,HomeTeam,Div,Result,Home_Goals,HomeWin,Season,Date,AwayTeam,Div,Result,Away_Goals,AwayWin
0,2011,2012-03-30,Aachen,D2,A,15,0,2011,2011-11-20,Aachen,D2,H,15,0
1,2011,2011-11-06,Augsburg,D1,A,20,0,2011,2011-08-27,Augsburg,D1,H,16,0
2,2011,2011-11-19,Bayern Munich,D1,A,49,0,2011,2012-03-31,Bayern Munich,D1,A,28,1
3,2011,2012-03-02,Bochum,D2,A,23,0,2011,2011-11-27,Bochum,D2,H,18,0
4,2011,2011-10-22,Braunschweig,D2,A,21,0,2011,2011-11-18,Braunschweig,D2,H,16,0


In [14]:
goals['Total_Goals'] = goals['Home_Goals'] + goals['Away_Goals']

goals.head()

,Season,Date,HomeTeam,Div,Result,Home_Goals,HomeWin,Season,Date,AwayTeam,Div,Result,Away_Goals,AwayWin,Total_Goals
0,2011,2012-03-30,Aachen,D2,A,15,0,2011,2011-11-20,Aachen,D2,H,15,0,30
1,2011,2011-11-06,Augsburg,D1,A,20,0,2011,2011-08-27,Augsburg,D1,H,16,0,36
2,2011,2011-11-19,Bayern Munich,D1,A,49,0,2011,2012-03-31,Bayern Munich,D1,A,28,1,77
3,2011,2012-03-02,Bochum,D2,A,23,0,2011,2011-11-27,Bochum,D2,H,18,0,41
4,2011,2011-10-22,Braunschweig,D2,A,21,0,2011,2011-11-18,Braunschweig,D2,H,16,0,37


# Total Wins

In [16]:
goals["TotalWin"] = df1["AwayWin"] + df["HomeWin"]

In [33]:
df['Total_Wins'] = df1['HomeWin'] + df2['AwayWin']

In [20]:
goals.head(20)

,Season,Date,HomeTeam,Div,Result,Home_Goals,HomeWin,Season,Date,AwayTeam,Div,Result,Away_Goals,AwayWin,Total_Goals,TotalWin
0,2011,2012-03-30,Aachen,D2,A,15,0,2011,2011-11-20,Aachen,D2,H,15,0,30,0
1,2011,2011-11-06,Augsburg,D1,A,20,0,2011,2011-08-27,Augsburg,D1,H,16,0,36,0
2,2011,2011-11-19,Bayern Munich,D1,A,49,0,2011,2012-03-31,Bayern Munich,D1,A,28,1,77,1
3,2011,2012-03-02,Bochum,D2,A,23,0,2011,2011-11-27,Bochum,D2,H,18,0,41,0
4,2011,2011-10-22,Braunschweig,D2,A,21,0,2011,2011-11-18,Braunschweig,D2,H,16,0,37,0
5,2011,2011-07-15,Cottbus,D2,H,18,1,2011,2012-02-06,Cottbus,D2,H,12,0,30,1
6,2011,2012-04-11,Dortmund,D1,H,44,1,2011,2011-08-27,Dortmund,D1,D,36,0,80,1
7,2011,2012-02-24,Dresden,D2,H,30,1,2011,2011-12-18,Dresden,D2,D,20,0,50,1
8,2011,2011-08-26,Duisburg,D2,H,23,1,2011,2011-08-07,Duisburg,D2,D,19,0,42,1
9,2011,2012-03-16,Ein Frankfurt,D2,H,38,1,2011,2011-11-07,Ein Frankfurt,D2,A,38,1,76,2


# Deleting repeated variables 

In [25]:
goals = goals[["Season", "Date", "Div", "HomeTeam", "AwayTeam", "Result", "Away_Goals", "Total_Goals", "TotalWin"]]

In [26]:
goals.head()

,Season,Season,Date,Date,Div,Div,HomeTeam,AwayTeam,Result,Result,Away_Goals,Total_Goals,TotalWin
0,2011,2011,2012-03-30,2011-11-20,D2,D2,Aachen,Aachen,A,H,15,30,0
1,2011,2011,2011-11-06,2011-08-27,D1,D1,Augsburg,Augsburg,A,H,16,36,0
2,2011,2011,2011-11-19,2012-03-31,D1,D1,Bayern Munich,Bayern Munich,A,A,28,77,1
3,2011,2011,2012-03-02,2011-11-27,D2,D2,Bochum,Bochum,A,H,18,41,0
4,2011,2011,2011-10-22,2011-11-18,D2,D2,Braunschweig,Braunschweig,A,H,16,37,0


In [30]:
goals.columns = ["Season1", "Season2", "Date1", "Date2", "Div1", "Div2", "TeamName1", "TeamName2", "Result1", "Result2",  "Away_Goals", "Total_Goals", "Total_Wins"]

In [31]:
del goals['Season2']
del goals['Div2']
del goals['TeamName2']

In [32]:
goals.head()

,Season1,Date1,Date2,Div1,TeamName1,Result1,Result2,Away_Goals,Total_Goals,Total_Wins
0,2011,2012-03-30,2011-11-20,D2,Aachen,A,H,15,30,0
1,2011,2011-11-06,2011-08-27,D1,Augsburg,A,H,16,36,0
2,2011,2011-11-19,2012-03-31,D1,Bayern Munich,A,A,28,77,1
3,2011,2012-03-02,2011-11-27,D2,Bochum,A,H,18,41,0
4,2011,2011-10-22,2011-11-18,D2,Braunschweig,A,H,16,37,0


# Total Losses 

# Home games 

In [22]:
cur.execute("""Select Season , Date , HomeTeam  ,Div, FTR as Result, sum(FTHG) As Home_Goals
               From Matches
               Where Season == '2011'
               and  Div != 'E0'
               Group By HomeTeam
;""")
df2 = pd.DataFrame(cur.fetchall())
df2.columns = [x[0] for x in cur.description]
df2.head()

,Season,Date,HomeTeam,Div,Result,Home_Goals
0,2011,2012-03-30,Aachen,D2,A,15
1,2011,2011-11-06,Augsburg,D1,A,20
2,2011,2011-11-19,Bayern Munich,D1,A,49
3,2011,2012-03-02,Bochum,D2,A,23
4,2011,2011-10-22,Braunschweig,D2,A,21


In [34]:
df2['HomeLoss'] = df2['Result'].map(lambda x: 0 if x == 'H' else 1)
df2.head()

,Season,Date,HomeTeam,Div,Result,Home_Goals,HomeLoss
0,2011,2012-03-30,Aachen,D2,A,15,1
1,2011,2011-11-06,Augsburg,D1,A,20,1
2,2011,2011-11-19,Bayern Munich,D1,A,49,1
3,2011,2012-03-02,Bochum,D2,A,23,1
4,2011,2011-10-22,Braunschweig,D2,A,21,1


In [35]:
cur.execute("""Select Season , Date , AwayTeam  ,Div, FTR as Result, sum(FTAG) As Away_Goals
               From Matches
               Where Season == '2011'
               and  Div != 'E0'
               Group By AwayTeam
;""")
df3 = pd.DataFrame(cur.fetchall())
df3.columns = [x[0] for x in cur.description]
df3.head()

,Season,Date,AwayTeam,Div,Result,Away_Goals
0,2011,2011-11-20,Aachen,D2,H,15
1,2011,2011-08-27,Augsburg,D1,H,16
2,2011,2012-03-31,Bayern Munich,D1,A,28
3,2011,2011-11-27,Bochum,D2,H,18
4,2011,2011-11-18,Braunschweig,D2,H,16


In [38]:
df3['AwayLoss'] = df3['Result'].map(lambda x: 1 if x == 'H' else 0)
df3.head()

,Season,Date,AwayTeam,Div,Result,Away_Goals,AwayLoss
0,2011,2011-11-20,Aachen,D2,H,15,1
1,2011,2011-08-27,Augsburg,D1,H,16,1
2,2011,2012-03-31,Bayern Munich,D1,A,28,0
3,2011,2011-11-27,Bochum,D2,H,18,1
4,2011,2011-11-18,Braunschweig,D2,H,16,1


In [ ]:
import numpy as np


win = df[df.TeamName == ‘Aachen’][‘Total_Wins’]
loss = df[df.TeamName == ‘Aachen’][‘Total_Losses’]
print(loss[0])
plt.bar([0,1],[loss[0],win[0]])
import numpy as np
fig = plt.figure(figsize=(10,10))
# Example data
